## Camera Calibration Process:

## 0. Preparation

In [1]:
import cv2
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob 
import matplotlib.pyplot as plt
import os
import json


In [2]:
# Auxiliar functions
def load_images(filenames: List) -> List:
    return [cv2.imread(filename) for filename in filenames]

def write_image(output_folder: str, img_name: str, img: np.array):
    img_path = os.path.join(output_folder, img_name)  
    os.makedirs(output_folder, exist_ok=True)
    cv2.imwrite(img_path, img)  

def show_image(img: np.array, img_name: str = "Image"):
    cv2.imshow(img_name, img)  
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

### Load the chessboard calibration images

In [3]:
imgs_gen_path = 'calibration_images/*.jpeg' 
imgs_path = sorted(glob.glob(imgs_gen_path))
imgs = load_images(imgs_path)  

imgs_path

['calibration_images\\WhatsApp Image 2025-12-23 at 21.18.44.jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.18.46.jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.18.58.jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.18.59 (1).jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.18.59.jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.19.05 (1).jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.19.05 (2).jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.19.05 (3).jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.19.05 (4).jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.19.05 (5).jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.19.05.jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.19.06 (1).jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.19.06 (2).jpeg',
 'calibration_images\\WhatsApp Image 2025-12-23 at 21.19.06 (3).jpeg',
 'calibration_images\\What

In [4]:
len(imgs)

66

### Detect the corners of the pattern and refine them

In [5]:
# Number of rows and cols
pattern_size = (11,8)

In [6]:
corners = [cv2.findChessboardCorners(
    cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), pattern_size) 
    for img in imgs]  

In [7]:
len(corners)  

66

In [9]:
corners[1][1].shape  

(88, 1, 2)

In [10]:
# Security copy
corners_copy = copy.deepcopy(corners)  

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01) 

# To refine corner detections we input grayscale images
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], pattern_size, (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

In [11]:
corners_refined[12].shape


(88, 1, 2)

In [12]:
len(corners_refined)

66

### Check that the detection process was succesful drawing the detected corners

In [13]:
imgs_copy = copy.deepcopy(imgs) 

In [14]:
output_folder_refined_corners = "refined_corners_output"

for i in range(len(corners_copy)):
    ret = corners_copy[i][0]
    if ret: 
        cv2.drawChessboardCorners(imgs_copy[i], pattern_size, corners_refined[i], ret) 
        #cv2.imshow(f"Img {i} and corners", imgs_copy[i]) 
        write_image(output_folder=output_folder_refined_corners, img_name=f"refined_corners_{i}.jpg", img=imgs_copy[i])
        cv2.waitKey(0)

cv2.destroyAllWindows()

In [16]:
def get_chessboard_points(chessboard_shape, dx, dy):

    cols, rows = chessboard_shape 

    grid_xy = np.mgrid[0:cols, 0:rows].T.reshape(-1,2)

    objp = np.zeros([grid_xy.shape[0], 3], dtype=np.float32) 

    objp[:,0] = grid_xy[:,0]*dx  
    objp[:,1] = grid_xy[:,1]*dy  

    return objp  

In [17]:
# Needs to be changed to the real position of our chessboard corners
dx, dy = 16, 16 
chessboard_points = [get_chessboard_points(pattern_size, dx, dy) for _ in corners_refined]  


In [18]:
len(chessboard_points)

66

In [19]:
chessboard_points[0]

array([[  0.,   0.,   0.],
       [ 16.,   0.,   0.],
       [ 32.,   0.,   0.],
       [ 48.,   0.,   0.],
       [ 64.,   0.,   0.],
       [ 80.,   0.,   0.],
       [ 96.,   0.,   0.],
       [112.,   0.,   0.],
       [128.,   0.,   0.],
       [144.,   0.,   0.],
       [160.,   0.,   0.],
       [  0.,  16.,   0.],
       [ 16.,  16.,   0.],
       [ 32.,  16.,   0.],
       [ 48.,  16.,   0.],
       [ 64.,  16.,   0.],
       [ 80.,  16.,   0.],
       [ 96.,  16.,   0.],
       [112.,  16.,   0.],
       [128.,  16.,   0.],
       [144.,  16.,   0.],
       [160.,  16.,   0.],
       [  0.,  32.,   0.],
       [ 16.,  32.,   0.],
       [ 32.,  32.,   0.],
       [ 48.,  32.,   0.],
       [ 64.,  32.,   0.],
       [ 80.,  32.,   0.],
       [ 96.,  32.,   0.],
       [112.,  32.,   0.],
       [128.,  32.,   0.],
       [144.,  32.,   0.],
       [160.,  32.,   0.],
       [  0.,  48.,   0.],
       [ 16.,  48.,   0.],
       [ 32.,  48.,   0.],
       [ 48.,  48.,   0.],
 

In [20]:
chessboard_points[0].shape

(88, 3)

### Obtain the calibration parameters for our camera

In [21]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]  

# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [22]:
valid_corners

array([[[[1342.0977 ,  766.6502 ]],

        [[1298.2479 ,  780.3515 ]],

        [[1254.0314 ,  794.20746]],

        ...,

        [[ 890.0215 ,  601.02826]],

        [[ 846.84094,  615.8752 ]],

        [[ 804.09436,  630.592  ]]],


       [[[1368.2789 ,  794.3445 ]],

        [[1322.1343 ,  809.29425]],

        [[1276.2126 ,  824.76953]],

        ...,

        [[ 892.97284,  622.7591 ]],

        [[ 849.4312 ,  638.8274 ]],

        [[ 805.59546,  655.14484]]],


       [[[1245.4803 ,  851.95294]],

        [[1198.2872 ,  843.06476]],

        [[1150.7062 ,  834.3657 ]],

        ...,

        [[ 922.5063 ,  459.92786]],

        [[ 877.1082 ,  453.27664]],

        [[ 831.44183,  446.0055 ]]],


       ...,


       [[[ 644.513  ,  678.4915 ]],

        [[ 676.83154,  682.4856 ]],

        [[ 709.21173,  685.7653 ]],

        ...,

        [[ 889.8174 ,  880.1773 ]],

        [[ 923.1162 ,  883.66284]],

        [[ 956.55054,  887.0669 ]]],


       [[[ 663.5202 ,  671.585  ]]

In [23]:
chessboard_points[0].shape

(88, 3)

In [24]:
valid_corners[0].shape

(88, 1, 2)

In [25]:
len(valid_corners)

64

In [26]:
len(chessboard_points)

66

In [ ]:
# We adjust the number of chessboard_points to be equal to the number of valid corners
chessboard_points = [chessboard_points[i] for i in range(len(valid_corners))]  
len(chessboard_points)

64

In [ ]:
img_size = (imgs_gray[0].shape[1], imgs_gray[0].shape[0]) 

rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(chessboard_points, valid_corners, img_size, None, None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs)) 

In [32]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("\nDistortion coefficients:\n", dist_coeffs)
print("\nRoot mean squared reprojection error:\n", rms)
print("\nExtrinsics:\n", extrinsics)

Intrinsics:
 [[1.47708757e+03 0.00000000e+00 9.86469883e+02]
 [0.00000000e+00 1.48148353e+03 6.72368302e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion coefficients:
 [[ 6.76789436e-02  5.27150448e-01 -3.06894709e-02  1.61476291e-03
  -2.46949075e+00]]

Root mean squared reprojection error:
 1.6166388760877022

Extrinsics:
 [array([[-9.36630524e-01, -3.47985391e-01,  4.03661989e-02,
         1.26204840e+02],
       [ 3.09172806e-01, -8.75297031e-01, -3.71843089e-01,
         3.42816169e+01],
       [ 1.64728377e-01, -3.35799457e-01,  9.27417536e-01,
         5.23598564e+02]]), array([[-9.29076695e-01, -3.63594134e-01,  6.79396786e-02,
         1.28911319e+02],
       [ 3.31423762e-01, -8.99859534e-01, -2.83568528e-01,
         4.19454078e+01],
       [ 1.64240021e-01, -2.40940087e-01,  9.56542257e-01,
         4.98389977e+02]]), array([[-9.67655025e-01,  1.40252881e-01,  2.09697121e-01,
         8.70014715e+01],
       [-1.71363463e-01, -9.75445396e-01, -1.38350436e-0

In [33]:
def save_calibration_data(mtx, dist, rvecs, tvecs, filename="calibration_data.json"):
    """
    Saves the intrinsic matrix, distortion coefficients, and extrinsic parameters
    (rvecs, tvecs) to a JSON file.
    """
    print("Saving calibration data...")
    
    # Convert NumPy arrays to standard Python lists
    if isinstance(mtx, np.ndarray):
        mtx_list = mtx.tolist()
    else:
        mtx_list = mtx

    if isinstance(dist, np.ndarray):
        # Flatten the distortion coefficients array before converting
        dist_list = np.ravel(dist).tolist() 
    else:
        dist_list = dist

    # Process Extrinsics (rvecs and tvecs are lists of arrays)
    # Convert each array within the list to a Python list
    rvecs_list = []
    if rvecs is not None:
        for r in rvecs:
            if isinstance(r, np.ndarray):
                rvecs_list.append(r.tolist())
            else:
                rvecs_list.append(r)

    tvecs_list = []
    if tvecs is not None:
        for t in tvecs:
            if isinstance(t, np.ndarray):
                tvecs_list.append(t.tolist())
            else:
                tvecs_list.append(t)

    # Create the data dictionary
    data = {
        "intrinsics": mtx_list,
        "distortion": dist_list,
        "extrinsics": {
            "rvecs": rvecs_list,
            "tvecs": tvecs_list
        }
    }

    # Write to file
    try:
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)
        print(f"Success! Data saved to '{filename}'")
        print(f"   Matrix: \n{np.array(mtx_list)}")
        print(f"   Distortion: {dist_list}")
        print(f"   Extrinsics saved: {len(rvecs_list)} vector pairs.")
    except Exception as e:
        print(f"Error saving the file: {e}")

if 'intrinsics' in locals() and 'dist_coeffs' in locals() and 'rvecs' in locals() and 'tvecs' in locals():
    save_calibration_data(intrinsics, dist_coeffs, rvecs, tvecs)
else:
    print("Saving error: Required calibration variables not found in local scope.")

Saving calibration data...
Success! Data saved to 'calibration_data.json'
   Matrix: 
[[1.47708757e+03 0.00000000e+00 9.86469883e+02]
 [0.00000000e+00 1.48148353e+03 6.72368302e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
   Distortion: [0.06767894363427608, 0.5271504479204613, -0.0306894708863236, 0.001614762907429389, -2.4694907498437453]
   Extrinsics saved: 64 vector pairs.
